# Meta Analyze PD Haplogroup Data
- **Author(s)** - Frank Grenn
- **Date Started** - June  2021
- **Quick Description:** meta analyze the major haplogroups logistic regression results using R

In [ ]:
library(metafor)
library(data.table)

In [ ]:
WRKDIR <- "$PATH/chrY"
PLOTDIR <- paste0(WRKDIR,"/meta_analysis_plots/major_haplogroups_covs_adjusted")

In [ ]:
data = fread(paste0(WRKDIR,"/log_reg_results_for_meta_newest.csv"))
print(dim(data))
print(head(data))

In [ ]:
#try per haplogroup
haplos <- unique(data$haplo_major)
print(haplos)

In [ ]:
results_df = data.frame()
for (haplo in haplos){
    haplo_data <- data[data$haplo_major == haplo,]
    print(haplo_data)


    meta_analysis_reg <- rma(yi=haplo_data$beta_logreg, sei = haplo_data$se_logreg)
    print(meta_analysis_reg)
    
    row = data.frame(beta=meta_analysis_reg$`beta`[,1], se = meta_analysis_reg$`se`, zval = meta_analysis_reg$`zval`, pval = meta_analysis_reg$pval, ci_lb = meta_analysis_reg$`ci.lb`, ci_ub = meta_analysis_reg$`ci.ub`)
    rownames(row) <- haplo

    results_df <- rbind(results_df, row)
}
print(results_df)

In [ ]:
#try per haplogroup but with no lbd cohort

results_df = data.frame()
for (haplo in haplos){
    haplo_data <- data[which(data$haplo_major == haplo & data$cohort!='amp_lbd_cc'),]
    #print(haplo_data)


    meta_analysis_reg <- rma(yi=haplo_data$beta_logreg, sei = haplo_data$se_logreg)
    #print(meta_analysis_reg)
    
    row = data.frame(beta=meta_analysis_reg$`beta`[,1], se = meta_analysis_reg$`se`, zval = meta_analysis_reg$`zval`, pval = meta_analysis_reg$pval, ci_lb = meta_analysis_reg$`ci.lb`, ci_ub = meta_analysis_reg$`ci.ub`)
    rownames(row) <- haplo

    results_df <- rbind(results_df, row)
}
print(results_df)

In [ ]:
#try per haplogroup but with no lbd cohort and a minimum sample count

results_df = data.frame()
for (haplo in haplos){
    haplo_data <- data[which(data$haplo_major == haplo),]
    #print(haplo_data)

    filter_haplo_data <- haplo_data[which(haplo_data$cohort!='amp_lbd_cc' & (haplo_data$cases+haplo_data$controls>=50)),]
    print(paste0("all datasets for ", haplo))
    print(haplo_data)
    print("remove lbd and anything with fewer than 50 samples with the haplogroup")
    print(filter_haplo_data)
    if(nrow(filter_haplo_data)!=0){
        #rma(yi=haplo_data$beta_logreg, sei = haplo_data$se_logreg,method = "FE")
        meta_analysis_reg <- rma(yi=filter_haplo_data$beta_logreg, sei = filter_haplo_data$se_logreg)
        print(meta_analysis_reg)
        png(file = paste0(PLOTDIR,"/",haplo,"_forest.png"),width = 480, height = 480)
        
        forest(meta_analysis_reg, slab = filter_haplo_data$cohort, main=paste0("Meta Analysis of Major Haplogroup ", haplo),atransf=exp,  mlab = "Fixed Effects", xlab = "Odds Ratio (95% CI) for major haplogroup", col="red", border = "red", cex=.9)
        dev.off()
        row = data.frame(beta=meta_analysis_reg$`beta`[,1], se = meta_analysis_reg$`se`, zval = meta_analysis_reg$`zval`, pval = meta_analysis_reg$pval, ci_lb = meta_analysis_reg$`ci.lb`, ci_ub = meta_analysis_reg$`ci.ub`)
        rownames(row) <- haplo

        results_df <- rbind(results_df, row)
    }

    

}

print(results_df)

In [ ]:
results_df$haplo_major <- rownames(results_df)
print(results_df)
write.csv(results_df, paste0(WRKDIR,"/meta_analyzed_major_haplo_newest.csv"), quote=FALSE,row.names=FALSE)